In [115]:
import pandas as pd
from nltk import word_tokenize
from collections import Counter
import gensim
import string
import numpy as np

In [2]:
data = pd.read_csv(r"C:\Users\alecr\Projects\assignments\Korbit_B\wikipedia_machine_learning.csv", sep='\t')

In [3]:
articles = data.apply(lambda x: x[2], axis=1)
titles = data.apply(lambda x: x[0], axis=1)
urls = data.apply(lambda x: x[1], axis=1)

In [4]:
articles = data.apply(lambda x: x[2], axis=1)
titles = data.apply(lambda x: x[0], axis=1)
urls = data.apply(lambda x: x[1], axis=1)
df = pd.concat([titles,urls,articles],axis=1)
df.columns = ['title', 'url','original_article']

In [5]:
df.dropna(inplace=True)

In [6]:
df.head()

,title,url,original_article
0,Outline of computer vision,https://en.wikipedia.org/wiki/Outline_of_compu...,The following outline is provided as an overvi...
1,Outline of natural language processing,https://en.wikipedia.org/wiki/Outline_of_natur...,The following outline is provided as an overvi...
2,Outline of robotics,https://en.wikipedia.org/wiki/Outline_of_robotics,The following outline is provided as an overvi...
3,Accuracy paradox,https://en.wikipedia.org/wiki/Accuracy_paradox,The accuracy paradox is the paradoxical findin...
4,Action model learning,https://en.wikipedia.org/wiki/Action_model_lea...,Action model learning(sometimes abbreviated ac...


In [7]:
df['url'][:4]

0    https://en.wikipedia.org/wiki/Outline_of_compu...
1    https://en.wikipedia.org/wiki/Outline_of_natur...
2    https://en.wikipedia.org/wiki/Outline_of_robotics
3       https://en.wikipedia.org/wiki/Accuracy_paradox
Name: url, dtype: object

In [8]:
pun_trans = str.maketrans(" ", ' ',string.punctuation)

# Clean titles

In [9]:
titles = df['title']

In [10]:
titles = titles.str.lower()

remove punctuation from titles

In [11]:
titles = titles.str.translate(pun_trans)

In [12]:
titles.head()

0                outline of computer vision
1    outline of natural language processing
2                       outline of robotics
3                          accuracy paradox
4                     action model learning
Name: title, dtype: object

In [13]:
titles = titles.apply(word_tokenize)

remove stopwords

In [14]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [15]:
titles = titles.apply(lambda x: [w for w in x if w not in stop_words])

In [16]:
titles.head()

0                 [outline, computer, vision]
1    [outline, natural, language, processing]
2                         [outline, robotics]
3                         [accuracy, paradox]
4                   [action, model, learning]
Name: title, dtype: object

In [17]:
c = Counter()

In [18]:
titles = titles.apply(c.update)

In [19]:
most_common = c.most_common(100)
#it's a list

In [20]:
common_words = [x[0] for x in most_common]

In [22]:
w2idx = {key: value for (value, key) in enumerate(common_words)}

In [23]:
w2idx.keys()

dict_keys(['list', 'analysis', 'theory', 'model', 'data', 'learning', 'software', 'algorithm', 'system', 'language', 'programming', 'information', 'computer', 'intelligence', 'network', 'problem', 'theorem', 'artificial', 'machine', 'science', 'design', 'function', 'statistics', 'management', 'neural', 'comparison', 'search', 'processing', 'computational', 'optimization', 'computing', 'space', 'regression', 'distribution', 'control', 'method', 'mathematics', 'systems', 'matrix', 'linear', 'engineering', 'process', 'semantic', 'random', 'statistical', 'probability', 'social', 'product', 'markov', 'clustering', 'law', 'image', 'paradox', 'error', 'graph', 'modeling', 'mathematical', 'quantum', 'tree', 'kernel', 'detection', 'research', 'bayesian', 'stochastic', 'transform', 'technology', 'robot', 'genetic', 'estimation', 'digital', 'game', 'test', 'code', 'decision', 'logic', 'architecture', 'marketing', 'recognition', 'cognitive', 'coding', 'robotics', 'selection', 'evolution', 'web', '

In [24]:
w2idx['natural']

86

In [25]:
idx2word = {v:k for (k,v) in w2idx.items()}

In [26]:
idx2word[2]

'theory'

In [27]:
df['title'] = titles

In [28]:
df.shape

(7316, 3)

In [29]:
num_instances = df.shape[0]

# clean articles

In [30]:
articles = df.original_article

In [31]:
articles = articles.str.lower()

In [32]:
articles = articles.str.translate(pun_trans)

In [33]:
articles = articles.apply(word_tokenize) 

In [34]:
articles = articles.apply(lambda x: [w for w in x if w not in stop_words])

In [35]:
articles.head()

0    [following, outline, provided, overview, topic...
1    [following, outline, provided, overview, topic...
2    [following, outline, provided, overview, topic...
3    [accuracy, paradox, paradoxical, finding, accu...
4    [action, model, learningsometimes, abbreviated...
Name: original_article, dtype: object

In [36]:
df['articles'] = articles

## get document embeddings somehow

In [37]:
article_length = articles.apply(lambda x: len(x))

In [38]:
article_length.describe()

count     7316.000000
mean      1077.317660
std       1207.922826
min          2.000000
25%        297.000000
50%        663.500000
75%       1382.000000
max      10713.000000
Name: original_article, dtype: float64

idea - truncate documents to the first 500 words, make a matrix with embeddings per word in the input, run a cnn on it, and train the classifier to recognize topics from abstract titles

In [39]:
from gensim import models, corpora

In [102]:
NUM_TOPICS = 10
MAX_TERMS = 50

In [41]:
dictionary = corpora.Dictionary(df.articles)

In [42]:
corpus = [dictionary.doc2bow(text) for text in df.articles]

In [43]:
def try_models(num_topics, current_corpus, current_dictionary):
    lda_model = models.LdaModel(corpus=corpus, num_topics=num_topics, id2word=current_dictionary)

    lsi_model = models.LsiModel(corpus=corpus, num_topics=num_topics, id2word=current_dictionary)
    
    return lda_model, lsi_model
    

In [124]:
lda_model, lsi_model = try_models(NUM_TOPICS, corpus, dictionary)

In [93]:
def get_term_lists(model, num_topics, max_terms):
    term_list = []
    for idx in range(num_topics):
        # Print the first 10 most representative topics
        s = model.print_topic(idx, max_terms)
        terms = s.split('"')[1::2]
        term_list.append(terms)
        #print("Topic #%s:" % idx, s.split('"')[1::2])
        #print(term_list[idx])
    return term_list

In [103]:
lda_list = get_term_lists(lda_model, NUM_TOPICS, MAX_TERMS)

In [104]:
lsi_list = get_term_lists(lsi_model, 10,50)

now I need to compare terms between the two

In [107]:
len(list(set(lda_list[0]) & set(lsi_list[0])))/MAX_TERMS

0.56

In [148]:
def mean_terms_in_common(list1, list2,num_topics):
    common = np.zeros(num_topics)
    for idx in range(num_topics):
        common[idx] = len(list(set(list1[idx]) & set(list2[idx])))
        #print("proportion of topics in common: ",common[idx])
    #print("mean: ", common.mean())
    
    return common.mean()

In [150]:
mean_terms_in_common(lda_list, lsi_list,2)

57.0

now to pipeline this

In [128]:
lda_model, lsi_model = try_models(NUM_TOPICS, corpus, dictionary)

In [135]:
lda_list = get_term_lists(lda_model, NUM_TOPICS, MAX_TERMS)
lsi_list = get_term_lists(lsi_model, NUM_TOPICS,MAX_TERMS)
mean_terms_in_common(lda_list, lsi_list,NUM_TOPICS)

0.296

going to try different numbers of topics:

In [137]:
num_topic_list = [5,8,10,15,20]
max_terms = 100

In [152]:
for n in num_topic_list:
    lda_model, lsi_model = try_models(n, corpus, dictionary)
    lda_list = get_term_lists(lda_model, n, max_terms)
    lsi_list = get_term_lists(lsi_model, n,max_terms)
    print("trying %d topics: " % n)
    print("there's a mean value of %d terms in common per topic between models" % mean_terms_in_common(lda_list, lsi_list,n))
    

trying 5 topics: 
there's a mean value of 38 terms in common between models
trying 8 topics: 
there's a mean value of 33 terms in common between models
trying 10 topics: 
there's a mean value of 32 terms in common between models
trying 15 topics: 
there's a mean value of 30 terms in common between models
trying 20 topics: 
there's a mean value of 27 terms in common between models
